In [21]:
import os
import subprocess
import cv2
import numpy as np

## Stope upravljanja kvalitetom - "rate control"

Osnovni termini:
* rate control - je aktivnost koju provodi video enkoder kada treba odlučiti koliko bitova uložiti na reprezentiranje određene sličice. Kod "manjkavog" (eng. lossy) kodiranja cilj je uštedjeti bitove (smanjiti veličinu datoteke), uz zadržavanja što više razine kvalitete. Na temelju ove aktivnosti definiran je omjer veličine i kvalitet. Zato je kvantizacijski parametar niži kod sličica više kvalitete (*veličina/kvaliteta*). 
* bitrate - omjer veličine video zapisa i njegovog trajanja - dva video zapisa iste duljine imaju različitu kvalitetu, ako jedan od njih ima viši bitrate
* macroblock - skupina pixela (npr. 16x16, 8x8, 4x4) koja je transformirana na temelju matematičkih funkcija prilikom upravljanja kvalitetom - tj. ti blokovi su zajedno komprimirani. Zato se kod jako komprimiranih video zapisa na sličicama vide "blokovi" (aka. "kvadratići")
* codec - je kratica za encoder/decoder - podatci generirani enkoderom(aka. kodirani podatci) uvijek mogu biti dekodirani odgovarajućim dekoderom. Obično su definirani odgovarajućim standardom (npr. H.264) koji kaže koji je format "bitstream"-a (sirovi podatci aka 0 i 1) kojeg daje enkoder. Koraci su: 1) enkoder prima video sličice i generira bitstream koji zadovoljava standard propisan codecom, 2) bitstream se zatim stavlja u "kontejner" (aka. format poput .mp4, .avi itd), koji je ništa drugo nego datoteka koja u sebi sadrži video i audio sirove podatke te ih sinkronizira. I sam kontejner(aka format) je također bitstream 3) dekoder preuzima bitstream u propisanom formatu i rekonstruira sličice. Stvari koje bune ljude su to da se npr. MPEG može odnositi i na codec (tj. standard) i na kontejner .mpeg.
* **libx264** je biblioteka koja implementira kodiranje, dekodiranja i transkodiranje video stream-ova (npr. iz jednog formata u drugi:.avi u .mp4) prema standardu *AVC/H.264/MPEG-4* 
* **ffmpeg** je softver koji koristim da bi radio manipulacije na video zapisima, te u sebi sadrži veći broj biblioteka poput libx264, libx265 itd.

Modovi upravljanja kvalitetom zapisa:

*Konstantna kvaliteta/varijabilni bitrate*

* **CRF** ("constant rate factor") - (0 - 51, niže je bolja kvaliteta tj. manja kompresija) - pokušava održati konstantnu kvalitetu kroz cijeli video zapis varirajući razinu kompresije. Ovdje se kvaliteta odnosi na ono što percipira ljudsko oko, a ne na kvalitetu izraženu mat. funkcijom. Stoga je razina kompresije veća kod sličica sa većom količinom akcije jer će kod takvih sličica ljudsko oko manje uočiti detalje, dok će kompresija biti manja kod "mirnih" sličica jer ovdje imamo više vremena uočiti razne detalje. Tehničkim rječnikom kvantizacijski parametar **QP** će biti niži kod mirnijih sličica, a viši kod sličica punih akcije. Kod primjene ovog moda **bitrate** može znatno varirati između zapisa iste duljine trajanja ovisno o sadržaju videa. Ako je CRF postavljen na 0 tada nema kompresije - "lossless" kodiranje. Većina codeca ovakava zapis ne može pročitati (N.B. ffmpeg može).

* **CQP** ("constant quantization parameter") - (0- 51, niže je bolja kvaliteta tj. manja kompresija) - svaka sličica komprimirana je na isti način, bez obzira na sadržaj. Zbog toga je ovdje kvaliteta svake sličice matematički ista, ali je moguće da je ljudsko oko različito percipira. Kvantizacijski parametar definira koliko bitova informacije ćemo izbaciti u određenom makrobloku, npr. ako je qp = 0 ("lossless") tada ne izbacujemo ništa, ali je datoteka velika. Također zbog toga **bitrate** jako varira.

*Konstantni bitrate* - CBR

* ovaj pristup ima smisla kada je bitno zadržati bitrate istim npr. ako imamo namjeru raditi stream na YouTube, pa si ne možemo dozvoliti "peak"-ove kod bitrate-a.
* ja sam koristio ovaj mod kod kodiranja "Ulazni_video" na svim zapisima uz bitrate = 2500. Iz razloga što su svi zapisi iste duljine, ovo osigurava i istu kvalitetu.
* ovaj mod je inače rasipan po pitanju veličine datoteke posebice ako postoje sličice koje je jednostavno kodirati, pa ne trebaju visok bitrate.

In [146]:
#Modovi upravljanja kvalitetom
#opcija koja kod ffmpeg-a za H.264 codec, osigurava istu kvalitetu, 
#ali utječe ne veličinu datoteke: sporo kodiranje => manja veličina
preset = "veryslow"  

In [143]:
path_to_video = "C:/Users/Public/V1.mp4"
out_dir = "C:/Users/Public"
out_file = "cut.mp4"

## 1. Zaključak:
Kod izrezivanja segmenata najbolje je koristit kopiranje bez re-kodiranja, jer je najbrže i kvaliteta ulaza ostaje. Iako je originalnom video dodano ispred i iza nekoliko crnih sličica, kod pretvorbe videa u sliku, te slike ffmpeg odbacuje.

In [145]:
#Kopiranja kod izrezivanja, a ne re-kodiranje
def video_to_segment_kopiranje(path_to_video, output_dir, output_file, start_point, duration):    
    with open(os.devnull, 'w') as ffmpeg_log: #os.devnull služi za prikupljanje svih printova i njegovo odbacivanje
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        ffmpeg_call = ["ffmpeg",            
                       "-ss", start_point,  
                       "-i", path_to_video, 
                       "-t", duration,      
                       "-c:v", "copy",   #definiramo samo kopiranje      
                       f"{output_dir}/{output_file}"]  
        subprocess.call(ffmpeg_call, stdout=ffmpeg_log, stderr=ffmpeg_log) 
        

In [88]:
#Dekodiranje i izrezivanje uz CRF = 1
def video_to_segment_kopiranje_crf_1(path_to_video, output_dir, output_file, start_point, duration):    
    with open(os.devnull, 'w') as ffmpeg_log: #os.devnull služi za prikupljanje svih printova i njegovo odbacivanje
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        ffmpeg_call = ["ffmpeg",            
                       "-ss", start_point,  
                       "-i", path_to_video, 
                       "-t", duration,      
                       "-crf", "1",   #ovdje radimo ponovno kodiranje    
                       f"{output_dir}/{output_file}"]  
        subprocess.call(ffmpeg_call, stdout=ffmpeg_log, stderr=ffmpeg_log) 

In [95]:
#Dekodiranje i izrezivanje uz CRF = 0
def video_to_segment_kopiranje_crf_0(path_to_video, output_dir, output_file, start_point, duration):    
    with open(os.devnull, 'w') as ffmpeg_log: #os.devnull služi za prikupljanje svih printova i njegovo odbacivanje
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        ffmpeg_call = ["ffmpeg",            
                       "-ss", start_point,  
                       "-i", path_to_video, 
                       "-t", duration,      
                       "-crf", "0",   #ovdje radimo ponovno kodiranje     
                       f"{output_dir}/{output_file}"]  
        subprocess.call(ffmpeg_call, stdout=ffmpeg_log, stderr=ffmpeg_log) 

In [125]:
video_to_segment_kopiranje(path_to_video_2, out_dir, out_file3, "9:18", "0:54")

In [89]:
video_to_segment_kopiranje_crf_1(path_to_video, out_dir, out_file, "9:18", "0:54")

In [96]:
video_to_segment_kopiranje_crf_0(path_to_video, out_dir, out_file, "9:18", "0:54")

In [144]:
cut_path = "C:/Users/Public/cut3.mp4"
out_image = "C:/Users/Public/Slike"
image_res = "320:192"
fps=5

## 2. Zaključak
Kod pretvorbe videa u sliku, potrebno je definirati -qscale, qmin i qmax - kvantizacijski parametri kvalitete, niže je bolje, 1 je najbolje.

In [112]:
def video_to_image_qs_1(path_to_video, output_dir, image_res, fps):
    with open(os.devnull, 'w') as ffmpeg_log: #os.devnull služi za prikupljanje svih printova i njegovo odbacivanje
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        ffmpeg_call = ["ffmpeg",            
                       "-i", path_to_video, 
                       "-vf", f"scale={image_res}", 
                       "-qscale:v", "1",    
                       "-qmin", "1",         
                       "-qmax", "1",        
                       "-r", f"{fps}",     
                       f"{output_dir}/slika-%04d.jpeg"] 
        subprocess.call(ffmpeg_call, stdout=ffmpeg_log, stderr=ffmpeg_log) 

In [151]:
video_to_image_qs_1(cut_path, out_image, image_res, fps)

## Analiza ponašanja piksela između slika

In [152]:
p1 = "C:/Users/Public/Slike/slika-0001.jpeg"
p2 = "C:/Users/Public/Slike/slika-0002.jpeg"
p3 = "C:/Users/Public/Slike/slika-0003.jpeg"

p4 = "C:/Users/Public/Slike/slika-0082.jpeg"
p5 = "C:/Users/Public/Slike/slika-0083.jpeg"
p6 = "C:/Users/Public/Slike/slika-0084.jpeg"

s1 = cv2.imread(p1)
s2 = cv2.imread(p2)
s3 = cv2.imread(p3)

s4 = cv2.imread(p4)
s5 = cv2.imread(p5)
s6 = cv2.imread(p6)

1 - kopiranje slika

In [86]:
np.sum(s1 == s2), np.sum(s2 == s3), np.sum(s3 == s1)

(80886, 62703, 56979)

In [87]:
np.sum(s4 == s5), np.sum(s5 == s6), np.sum(s6 == s4)

(44024, 50739, 39534)

2 - rekodiranje uz crf = 1

In [93]:
np.sum(s1 == s2), np.sum(s2 == s3), np.sum(s3 == s1)

(85817, 62489, 56799)

In [94]:
np.sum(s4 == s5), np.sum(s5 == s6), np.sum(s6 == s4)

(43600, 50110, 38678)

3 - rekodiranje uz crf = 0

In [101]:
np.sum(s1 == s2), np.sum(s2 == s3), np.sum(s3 == s1)

(80886, 62703, 56979)

In [102]:
np.sum(s4 == s5), np.sum(s5 == s6), np.sum(s6 == s4)

(44024, 50739, 39534)

4 - uz drugačiji video scale - kopiranje

In [153]:
np.sum(s1 == s2), np.sum(s2 == s3), np.sum(s3 == s1)

(79134, 54896, 49840)

In [154]:
np.sum(s4 == s5), np.sum(s5 == s6), np.sum(s6 == s4)

(51161, 53390, 44400)